<a href="https://colab.research.google.com/github/Ramsiit2010/OCIGenAIBot/blob/main/RCOE_OCIGenAIBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Run Your GitHub Flask App on Google Colab (with Public URL)

**This notebook will:**
1) Clone your GitHub repo (OCIGenAIBot - RCOE Multi-Agent Architecture)
2) Install Python requirements
3) Run your Flask app via Gunicorn on port 5000
4) Expose a public HTTPS URL using ngrok

**Features:**
- MCP (Model Context Protocol) architecture with 5 specialized agents
- Dual-Mode General Agent: ORDS NL2SQL + OCI Gen AI Inference
- Region: us-ashburn-1 (default, configurable)

➡️ Just work top-to-bottom. Fill in your repo/config in the next cell.

In [ ]:
REPO_URL = "https://github.com/Ramsiit2010/OCIGenAIBot.git"  # <-- put your repo URL
PROJECT_DIR_NAME = "OCIGenAIBot"  # folder name after clone (edit if different)

# If your app exposes a Flask instance named `app`, set it here as module:variable
# Examples:
#   Simple app at repo root in app.py:        APP_MODULE = "app:app"
#   Package layout myapp/__init__.py -> app:  APP_MODULE = "myapp:app"
#   run.py that creates app variable:         APP_MODULE = "run:app"
APP_MODULE = "AskMeChatBot:app"  # <-- change to match your entry point

# Optional: If your repo needs environment variables, set them here
EXTRA_ENVS = {
    # "FLASK_ENV": "development",
    # "SOME_API_KEY": "",
}

# 🔐 (Optional but recommended) Your ngrok Authtoken for better reliability
# Get one free at https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTHTOKEN = "35KIlIDMzheum8MLvD4kqv39A4q_RD1X1u3CM4MhztuXkadu"  # paste token here or leave blank

In [ ]:
%%bash
python -V
pip install --upgrade pip >/dev/null
pip install flask gunicorn pyngrok >/dev/null
echo "✅ Installed: flask, gunicorn, pyngrok"

In [ ]:
import os, shutil, subprocess, sys
import stat

def remove_readonly(func, path, excinfo):
    """Error handler for shutil.rmtree to handle read-only files on Windows"""
    os.chmod(path, stat.S_IWRITE)
    func(path)

if os.path.exists(PROJECT_DIR_NAME):
    shutil.rmtree(PROJECT_DIR_NAME, onerror=remove_readonly)

print("Cloning repo...", REPO_URL)
rc = subprocess.call([sys.executable, "-m", "pip", "install", "gitpython"], stdout=subprocess.DEVNULL)
from git import Repo
Repo.clone_from(REPO_URL, PROJECT_DIR_NAME)
print("✅ Repo cloned into:", PROJECT_DIR_NAME)


In [ ]:
from google.colab import files
print('Upload ONLY the secret files: .env and oci_api_key.pem')
uploaded = files.upload()
saved = []
for name, data in uploaded.items():
    if name not in ('.env', 'oci_api_key.pem'):
        print(f'Skipping unexpected file: {name}')
        continue
    with open(name, 'wb') as f:
        f.write(data)
    print(f'Saved: {name}')
    saved.append(name)
print('Saved files:', saved)

In [ ]:
import os, subprocess, sys
req_path = os.path.join(PROJECT_DIR_NAME, "requirements.txt")
if os.path.exists(req_path):
    print("Installing requirements.txt...")
    rc = subprocess.call([sys.executable, "-m", "pip", "install", "-r", req_path])
    if rc != 0:
        raise RuntimeError("pip install -r requirements.txt failed")
    print("✅ Requirements installed")
else:
    print("⚠️ No requirements.txt found — skipping")


In [ ]:
import os
for k, v in EXTRA_ENVS.items():
    os.environ[k] = str(v)
print("Set envs:", list(EXTRA_ENVS.keys()))


In [ ]:
import os, importlib, sys
sys.path.insert(0, PROJECT_DIR_NAME)

module_name, var_name = APP_MODULE.split(":", 1)
print(f"🔎 Importing {module_name}.{var_name} ...")
mod = importlib.import_module(module_name)
assert hasattr(mod, var_name), f"Could not find '{var_name}' in module '{module_name}'. Update APP_MODULE."
print("✅ Flask app reference found.")


In [ ]:
import os, subprocess, sys, time, threading
from pathlib import Path

PORT = 5000

# Ensure APP_MODULE and PROJECT_DIR_NAME are defined (from cell 2)
if 'APP_MODULE' not in globals() or 'PROJECT_DIR_NAME' not in globals():
    raise RuntimeError("❌ APP_MODULE or PROJECT_DIR_NAME not found. Please run cell 2 first.")

# Import the Flask app
sys.path.insert(0, PROJECT_DIR_NAME)
module_name, var_name = APP_MODULE.split(":", 1)

import importlib
mod = importlib.import_module(module_name)
flask_app = getattr(mod, var_name)

# Run Flask in a background thread (since we're on Windows)
def run_flask():
    flask_app.run(host='0.0.0.0', port=PORT, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()
time.sleep(2)

print(f"✅ Flask server started on port {PORT}")
print("📄 Server is running in background thread")


In [ ]:
import time
LOGFILE = "/tmp/gunicorn.log"
print("--- Gunicorn Logs (last 200 lines) ---")
try:
    with open(LOGFILE, 'r') as f:
        lines = f.readlines()[-200:]
        for line in lines:
            print(line, end='')
except FileNotFoundError:
    print("No logs yet.")


In [ ]:
%pip install --upgrade pyngrok
from pyngrok import ngrok, conf
import time

if NGROK_AUTHTOKEN:
    conf.get_default().auth_token = NGROK_AUTHTOKEN
    print("🔐 ngrok authtoken set")
else:
    print("ℹ️ No ngrok authtoken set — using anonymous session")

print("Starting ngrok tunnel to 5000...")
try:
    public_tunnel = ngrok.connect(5000, proto="http")
    public_url = public_tunnel.public_url
    print("🌍 Public URL:", public_url)
    print("Open this in your browser. Keep this notebook running.")
except Exception as e:
    print(f"❌ Error starting ngrok: {e}")
    print("💡 Alternative: Install ngrok manually from https://ngrok.com/download")


: 

## ✅ You're Live!

- Your app is running on port **5000** inside Colab.
- ngrok is exposing a **Public URL** printed above.
- **Keep this notebook open**; if the runtime sleeps, the URL will stop.

### Application Features
- **5 MCP Agents**: General (dual-mode), Finance, HR, Orders, Reports
- **Dual-Mode General Agent**:
  - Database queries → ORDS NL2SQL (no credentials needed)
  - Knowledge questions → OCI Gen AI (us-ashburn-1)
  - Intelligent keyword detection + fallback chain
- **Pure Gen AI Intent Detection** (force mode)

### Troubleshooting
- If you see `ModuleNotFoundError`, make sure `requirements.txt` includes all libs and re-run the *Install requirements* cell.
- If Gunicorn can't import your app, double-check `APP_MODULE` in the config cell.
- Use the **Tail logs** cell to see server errors.
- Ensure `.env` and `oci_api_key.pem` are in the project folder.

For architecture details: [GENERAL_AGENT_ARCHITECTURE.md](https://github.com/Ramsiit2010/OCIGenAIBot/blob/main/GENERAL_AGENT_ARCHITECTURE.md)